# Onion peeling procedure

## Input and output

We want to compute the array:
$$
\forall (k, l) \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket^2, \, \widehat{I}_D(k, l) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{2ku + 2lv}{m} \right)
$$

from the samples of the pseudo polar Fourier transform of $I$:
$$
\widehat{I}_h (k, l) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{-\frac{2lk}{n}u + kv}{m} \right) \\
\widehat{I}_v (k, l) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{ku -\frac{2lk}{n}v}{m} \right)
$$

where $k \in \llbracket -n, n \rrbracket$ and $l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket$. These arrays are basically samples of the trigonometric polynomial:
$$
\widehat{I}(\xi_x, \xi_y) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{\xi_x u + \xi_y v}{m} \right)
$$

## Description of the algorithm

At step $k \in \llbracket -n/2, 0 \rrbracket$, suppose we have computed $\widehat{I}_D(k', :), \widehat{I}_D(-k', :), \widehat{I}_D(:, k'), \widehat{I}_D(:, -k')$ for all $k' < k$.

### Horizontal lines

We want to compute $\widehat{I}_D(k, :)$. It is:
$$
\begin{align*}
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, \widehat{I}_D(k, l) &= \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{2ku}{m} \right) \exp\left(-2i\pi \frac{2lv}{m} \right) \\
&= P\left( -\frac{4\pi l}{m} \right)
\end{align*}
$$

where:
$$
P(x) = \sum_{v \in \mathcal{D}(n)} \left[\sum_{u \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{2ku}{m} \right) \right] \exp\left(i v x \right) = \sum_{v \in \mathcal{D}(n)} \alpha_v \exp\left(i v x \right)
$$

$P$ is a univariate trigonometric polynomial. If we look at $\widehat{I}_v(2k, :)$, we see that:
$$
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, \widehat{I}_v(2k, l) = P\left(\frac{8\pi l k}{n m} \right)
$$

We also have already computed some values of $\widehat{I}_D$. If $k \leq 0$, we have for $k' < k$:
$$
\widehat{I}_D(k, k') = P\left( -\frac{4\pi k'}{m} \right) \text{ and } \widehat{I}_D(k, -k') = P\left(\frac{4\pi k'}{m} \right)
$$

If $k \geq 0$, we have for $k' > k$:
$$
\widehat{I}_D(k, k') = P\left( -\frac{4\pi k'}{m} \right) \text{ and } \widehat{I}_D(k, -k') = P\left(\frac{4\pi k'}{m} \right)
$$

The problem we need to solve is the following: from the samples of $P$ we already know, resample $P$ at the target positions $- 4 \pi l / m$ for $l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket$. 

This is done by using fast resampling algorithms, which allows us to compute the coefficients $\alpha_v$. Then, we need to compute:
$$
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, P\left( -\frac{4\pi l}{m} \right) = \sum_{v \in \mathcal{D}(n)} \alpha_v \exp\left(- 2 i \pi \frac{2 l v}{m} \right)
$$

which is the 1D FFT of a zero-padded version of $\alpha$, sampled at $l' = 2l$ for $l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket$.

In the article, since we want to compute $n$ values of $P$ (they exclude the first/last row since it is already known), they first select the $n$ values we know that are the closest to the target positions. We don't know if this is necessary, as the fast resampling works when we have more known points than target ones. The number of known points is:
$$
n + 1 + 2s
$$

where $s \geq 0$ is the step.

In [677]:
import numpy as np
import matplotlib.pyplot as plt

from ppfft import ppfft
from pad import pad
from new_fft import new_fft, new_fft2
from fast_resampling import compute_alpha


# Checking PPFFT

In [678]:
def slow_DTFT(a, omega_x, omega_y, m):
    n, _ = a.shape
    u = np.arange(-n//2, n//2)
    v = np.arange(-n//2, n//2)
    return np.einsum("uv,u,v->", a, np.exp(-2j * np.pi * omega_x * u / m), np.exp(-2j * np.pi * omega_y * v / m))


In [679]:
n = 20
im = np.random.rand(n, n)
hori_ppfft, vert_ppfft = ppfft(im)


In [680]:
err = []

for i in range(2*n+1):
    for j in range(n+1):
        k = -n + i
        l = -(n//2) + j
        err.append(hori_ppfft[l + n//2, k + n] -
                   slow_DTFT(im, -2 * l * k / n, k, 2 * n + 1))

print(np.max(np.abs(err)))

err = []

for i in range(2*n+1):
    for j in range(n+1):
        k = -n + i
        l = -(n//2) + j
        err.append(vert_ppfft[l + n//2, k + n] -
                   slow_DTFT(im, k, -2 * l * k / n, 2 * n + 1))

print(np.max(np.abs(err)))


3.972214585701295e-06
3.972214585701295e-06


The horizontal ppfft is stored in ``hori_ppfft`` which is a $(n + 1, 2n + 1)$ array satisfying:
$$
\forall (k, l) \in \llbracket -n, n \rrbracket \times \llbracket -n/2, n/2 \rrbracket, \, \widehat{I}_h(k, l) = \texttt{hori\_ppfft[l + n//2, k + n]}
$$

The vertical ppfft is stored in ``vert_ppfft`` which is a $(n + 1, 2n + 1)$ array satisfying:
$$
\forall (k, l) \in \llbracket -n, n \rrbracket \times \llbracket -n/2, n/2 \rrbracket, \, \widehat{I}_v(k, l) = \texttt{vert\_ppfft[l + n//2, k + n]}
$$

## Computation of true $\widehat{I}_D$

In [681]:
def compute_true_Id(im):
    n = len(im)
    m = 2 * n + 1
    pad_im = pad(im, (m, m))
    return new_fft2(pad_im)[::2, ::2]


In [682]:
def test_true_Id(im):
    half_n = len(im) // 2
    m = 2 * len(im) + 1
    v = np.arange(-half_n, half_n)
    u = np.arange(-half_n, half_n)
    l_prime = 2 * np.arange(-half_n, half_n + 1)
    k_prime = 2 * np.arange(-half_n, half_n + 1)
    v_l_prime = np.einsum("v,l->vl", v, l_prime)
    u_k_prime = np.einsum("u,k->uk", u, k_prime)
    return np.einsum("uv,vl,uk->kl", im, np.exp(-2j * np.pi * v_l_prime / m), np.exp(-2j * np.pi * u_k_prime / m))


In [683]:
true_Id = compute_true_Id(im)
np.max(np.abs(true_Id - test_true_Id(im)))


1.7053025658242404e-13

## Initialization of $\widehat{I}_D$

We have, for all $l \in \llbracket -n/2, n/2\rrbracket$:
$$
\begin{align*}
&\widehat{I}_D(-n/2, l) &= \widehat{I}_v(-n, l) &= \texttt{vert\_ppfft[l + n//2, 0]} \\
&\widehat{I}_D(n/2, l) &= \widehat{I}_v(n, -l) &= \texttt{vert\_ppfft[-l + n//2, -1]}\\
&\widehat{I}_D(l, -n/2) &= \widehat{I}_h(-n, l) &= \texttt{hori\_ppfft[l + n//2, 0]}\\
&\widehat{I}_D(l, n/2) &= \widehat{I}_h(n, -l) &= \texttt{hori\_ppfft[-l + n//2, -1]}
\end{align*}
$$

In [684]:
def initialize(hori_ppfft, vert_ppfft):
    n = hori_ppfft.shape[0] - 1

    I_d = np.zeros(shape=(n+1, n+1), dtype=complex)

    I_d[0] = vert_ppfft[:, 0]  # x = -n/2
    I_d[-1] = vert_ppfft[::-1, -1]  # x = n/2
    I_d[:, 0] = hori_ppfft[:, 0]  # y = -n/2
    I_d[:, -1] = hori_ppfft[::-1, -1]  # y = n/2

    return I_d


In [685]:
hori_ppfft, vert_ppfft = ppfft(im)
Id = initialize(hori_ppfft, vert_ppfft)


In [686]:
print(np.max(np.abs(Id[0] - true_Id[0])))
print(np.max(np.abs(Id[-1] - true_Id[-1])))
print(np.max(np.abs(Id[:, 0] - true_Id[:, 0])))
print(np.max(np.abs(Id[:, -1] - true_Id[:, -1])))


2.719179572996309e-07
2.719179572996309e-07
3.703973260487359e-07
3.703973260487359e-07


# Resampling from $\alpha$

In [687]:
def resample_row(alpha):
    n = len(alpha)
    pad_alpha = pad(alpha, new_shape=(2 * n + 1,))
    fft_alpha = new_fft(pad_alpha)
    return fft_alpha[::2]


In [688]:
def test_resample_alpha(alpha):
    half_n = len(alpha) // 2
    v = np.arange(-half_n, half_n)
    l_prime = 2 * np.arange(-half_n, half_n + 1)
    v_l_prime = np.einsum("v,l->vl", v, l_prime)
    return np.einsum("v,vl->l", alpha, np.exp(-2j * np.pi * v_l_prime / (2 * len(alpha) + 1)))


In [689]:
alpha = np.random.rand(10)
np.max(np.abs(resample_row(alpha) - test_resample_alpha(alpha)))


1.47313877870876e-15

# Recovering a row

In [690]:
def true_alpha_hori(k, im):
    n = len(im)
    half_n = n//2
    u = np.arange(-half_n, half_n)
    return np.einsum("uv,u->v", im, np.exp(-2j * np.pi * k * 2 * u / (2 * n + 1)))


In [691]:
def select_n_closest(y):
    pass


In [692]:
def recover_row_test(k: int, vert_ppfft, I_D):
    """
    Recovers row k [| -n/2 + 1, n/2 - 1 |] of I_D
    """
    n, m = vert_ppfft.shape[0] - 1, vert_ppfft.shape[1]
    half_n = n//2
    true_k = k + half_n

    known_ppfft = vert_ppfft[:, 2 * k + n]  # n + 1 elements
    y_ppfft = 8 * np.pi * k * np.arange(-half_n, half_n + 1) / (n * m)

    if k <= 0:
        known_I_D_left = I_D[true_k, :true_k]
        y_left = - 4 * np.pi * np.arange(-half_n, k) / m

        known_I_D_right = I_D[true_k, -true_k:]
        y_right = 4 * np.pi * np.arange(-half_n, k) / m
    else:
        known_I_D_right = I_D[true_k, :(n - true_k)]
        y_left = - 4 * np.pi * np.arange(k + 1, half_n + 1) / m

        known_I_D_left = I_D[true_k, (true_k - n):]
        y_right = 4 * np.pi * np.arange(k + 1, half_n + 1) / m

    known_samples = np.concatenate(
        (known_I_D_left, known_ppfft, known_I_D_right))

    y = np.concatenate((y_left, y_ppfft, y_right))

    alpha = compute_alpha(y, n, known_samples)

    res = resample_row(alpha)

    print(res[0] - I_D[true_k, 0])
    print(res[-1] - I_D[true_k, -1])

    I_D[true_k] = res

    return alpha


In [693]:
n = 20
im = np.random.rand(n, n)
hori_ppfft, vert_ppfft = ppfft(im)
Id = initialize(hori_ppfft, vert_ppfft)

k = -(n//2) + 1
alpha = recover_row_test(k, vert_ppfft, Id)
true = true_alpha_hori(k, im)
print(np.max(np.abs(true - alpha)))

k *= -1
alpha = recover_row_test(k, vert_ppfft, Id)
true = true_alpha_hori(k, im)
print(np.max(np.abs(true - alpha)))


(8.394865469441015e-08-9.930555044945777e-09j)
(-6.647786054969629e-08+1.5634522121388272e-08j)
5.232427091358264e-08
(-6.647786054969629e-08-1.5634522121388272e-08j)
(8.394865469441015e-08+9.930555044945777e-09j)
5.232427091358264e-08


# Recovering a column

At step $k \in \llbracket -n/2, 0 \rrbracket$, suppose we have computed $\widehat{I}_D(k', :), \widehat{I}_D(-k', :), \widehat{I}_D(:, k'), \widehat{I}_D(:, -k')$ for all $k' < k$.

We want to compute $\widehat{I}_D(:, k)$. It is:
$$
\begin{align*}
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, \widehat{I}_D(l, k) &= \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{2lu}{m} \right) \exp\left(-2i\pi \frac{2kv}{m} \right) \\
&= P\left( -\frac{4\pi l}{m} \right)
\end{align*}
$$

where:
$$
P(x) = \sum_{u \in \mathcal{D}(n)} \left[\sum_{v \in \mathcal{D}(n)} I(u, v) \exp\left(-2i\pi \frac{2kv}{m} \right) \right] \exp\left(i u x \right) = \sum_{u \in \mathcal{D}(n)} \alpha_v \exp\left(i u x \right)
$$

$P$ is a univariate trigonometric polynomial. If we look at $\widehat{I}_h(2k, :)$, we see that:
$$
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, \widehat{I}_h(2k, l) = P\left(\frac{8\pi l k}{n m} \right)
$$

We also have already computed some values of $\widehat{I}_D$. If $k \leq 0$, we have for $k' < k$:
$$
\widehat{I}_D(k', k) = P\left( -\frac{4\pi k'}{m} \right) \text{ and } \widehat{I}_D(-k', k) = P\left(\frac{4\pi k'}{m} \right)
$$

If $k \geq 0$, we have for $k' > k$:
$$
\widehat{I}_D(k', k) = P\left( -\frac{4\pi k'}{m} \right) \text{ and } \widehat{I}_D(-k', k) = P\left(\frac{4\pi k'}{m} \right)
$$

The problem we need to solve is the following: from the samples of $P$ we already know, resample $P$ at the target positions $- 4 \pi l / m$ for $l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket$. 

This is done by using fast resampling algorithms, which allows us to compute the coefficients $\alpha_v$. Then, we need to compute:
$$
\forall l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket, \, P\left( -\frac{4\pi l}{m} \right) = \sum_{v \in \mathcal{D}(n)} \alpha_v \exp\left(- 2 i \pi \frac{2 l v}{m} \right)
$$

which is the 1D FFT of a zero-padded version of $\alpha$, sampled at $l' = 2l$ for $l \in \llbracket - \frac{n}{2}, \frac{n}{2}\rrbracket$.

In [694]:
def true_alpha_vert(k, im):
    n = len(im)
    half_n = n//2
    v = np.arange(-half_n, half_n)
    return np.einsum("uv,v->u", im, np.exp(-2j * np.pi * k * 2 * v / (2 * n + 1)))


In [695]:
def recover_col_test(k: int, hori_ppfft, I_D):
    """
    Recovers row k [| -n/2 + 1, n/2 - 1 |] of I_D
    """
    n, m = hori_ppfft.shape[0] - 1, hori_ppfft.shape[1]
    half_n = n//2
    true_k = k + half_n

    known_ppfft = hori_ppfft[:, 2 * k + n]  # n + 1 elements
    y_ppfft = 8 * np.pi * k * np.arange(-half_n, half_n + 1) / (n * m)

    if k <= 0:
        known_I_D_left = I_D[:true_k, true_k]
        y_left = - 4 * np.pi * np.arange(-half_n, k) / m

        known_I_D_right = I_D[-true_k:, true_k]
        y_right = 4 * np.pi * np.arange(-half_n, k) / m
    else:
        known_I_D_right = I_D[:(n - true_k), true_k]
        y_left = - 4 * np.pi * np.arange(k + 1, half_n + 1) / m

        known_I_D_left = I_D[(true_k - n):, true_k]
        y_right = 4 * np.pi * np.arange(k + 1, half_n + 1) / m

    known_samples = np.concatenate(
        (known_I_D_left, known_ppfft, known_I_D_right))

    y = np.concatenate((y_left, y_ppfft, y_right))

    alpha = compute_alpha(y, n, known_samples)

    res = resample_row(alpha)

    print(res[0] - I_D[0, true_k])
    print(res[-1] - I_D[-1, true_k])

    I_D[true_k] = res

    return alpha


In [696]:
n = 20
im = np.random.rand(n, n)
hori_ppfft, vert_ppfft = ppfft(im)
Id = initialize(hori_ppfft, vert_ppfft)

k = -(n//2) + 1
alpha = recover_col_test(k, hori_ppfft, Id)
true = true_alpha_vert(k, im)
print(np.max(np.abs(true - alpha)))

k *= -1
alpha = recover_col_test(k, hori_ppfft, Id)
true = true_alpha_vert(k, im)
print(np.max(np.abs(true - alpha)))


(-7.066727203408618e-09-6.546620490155419e-08j)
(-6.503597660412197e-11+5.90204063399824e-08j)
5.172929352139665e-08
(-6.503597660412197e-11-5.90204063399824e-08j)
(-7.066727203408618e-09+6.546620490155419e-08j)
5.172929352139665e-08


In [697]:
import matplotlib.pyplot as plt


# Note pour moi-même

Si $n$ points suffisent pour re-sample le polynôme trigonométrique, alors pas besoin d'attendre que les premiers points soient calculés pour calculer les suivants, on peut tout faire en parallèle.